In [59]:
import numpy as np
from dataset.mnist import load_mnist
import sys
np.set_printoptions(threshold=sys.maxsize)

# im2col method

입력 받은 이미지를 2차원 배열로 변환

In [75]:
def im2col(input_data, filter_h, filter_w, stride=1,pad=0):
    """
    파라미터
    input_data : 4차원 배열(이미지수, 채널수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    """
    
    N,C,H,W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride+1
    out_w = (W + 2*pad - filter_w)//stride+1

    img = np.pad(input_data, [(0,0),(0,0),(pad,pad),(pad,pad)],'constant')
    col = np.zeros((N,C,filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:,:,y,x,:,:] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    col = col.transpose(0,4,5,1,2,3).reshape(N*out_h * out_w, -1)
    return col

# col2im method

In [5]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """
    파라미터
    col : 2차원배열
    input_shape : 원래 이미지 데이터의 형상
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    """
    
    N,C,H,W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    
    col = col.reshape(N,out_h, out_w, C,filter_h,filter_w).transpose(0,3,4,5,1,2)
    img = np.zeros((N,C, H + 2*pad + stride-1,W + 2*pad + stride-1))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:,:,y,x,:,:] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    return img[:,:,pad:pad+H,pad:pad+W]

# Conv2D layer

In [7]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        """
        FN : 필터의 개수
        C : 필터의 깊이
        FH : 필터의 높이
        FW : 필터의 너비
        """
        
        N, C, H, W = x.shape
        """
        N : 입력 데이터의 개수
        C : 입력 데이터의 깊이
        H : 입력 데이터의 높이
        W : 입력 데이터의 너비
        """
        
        
        out_h = (H + 2*self.pad - FH)//self.stride + 1
        out_w = (H + 2*self.pad - FW)//self.stride + 1
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        
        out = np.dot(col,col_W) + self.b
        
        out = out.reshape(N, out_h,out_w,-1).transpose(0,3,1,2)
        
        return out

# Pooling layer

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        N,C,H,W = x.shape
        out_h = (H-self.pool_h) // self.stride + 1
        out_w = (H-self.pool_w) // self.stride + 1
        
        col = im2col(x,self.pool_h, self.pool_w,self.stride, self.pad)
        out = np.max(col,axis=1)
        col = col.reshape(-1,self.pool_h * self.pool_w)
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        
        return out

# Flatten layer

In [9]:
class Flatten:
    def __init__(self, x):
        self._shape = ()
        
    def forward(self,prev):
        self._shape = prev.shape
        return np.ravel(prev).reshape(prev.shape[0],-1)

### 이미지값 생성

In [77]:
X = np.random.rand(1,1,8,8)
print(X,"\n")
print(X.shape)
print(X[0].shape)
print(X[0][0])

[[[[0.8624802  0.4237964  0.40930859 0.17589646 0.29971498 0.86416746
    0.04815052 0.66181075]
   [0.05338908 0.3669698  0.37207691 0.82215844 0.52263086 0.59655656
    0.31322287 0.02934578]
   [0.86376853 0.99019456 0.26679653 0.97749604 0.8854396  0.91710142
    0.12305989 0.24030879]
   [0.67939017 0.27623327 0.88779469 0.70786682 0.40738597 0.31880154
    0.62366982 0.52629316]
   [0.19861288 0.20548148 0.67177928 0.72766764 0.86794151 0.94940576
    0.57408924 0.34056771]
   [0.16043901 0.02841533 0.20803795 0.16396695 0.28622979 0.04422877
    0.46578606 0.28088837]
   [0.58138855 0.68392959 0.34952271 0.94532829 0.9134112  0.14910784
    0.0327075  0.70354008]
   [0.13989748 0.51181935 0.06181297 0.29707818 0.53505348 0.99826314
    0.12847647 0.12801574]]]] 

(1, 1, 8, 8)
(1, 8, 8)
[[0.8624802  0.4237964  0.40930859 0.17589646 0.29971498 0.86416746
  0.04815052 0.66181075]
 [0.05338908 0.3669698  0.37207691 0.82215844 0.52263086 0.59655656
  0.31322287 0.02934578]
 [0.863768

In [78]:
X.shape[0]

1

### filter 생성

In [81]:
w = np.random.randn(1,1,2,2)
col = im2col(X,w.shape[2],w.shape[3], stride=1,pad=0)
print(col)

[[0.8624802  0.4237964  0.05338908 0.3669698 ]
 [0.4237964  0.40930859 0.3669698  0.37207691]
 [0.40930859 0.17589646 0.37207691 0.82215844]
 [0.17589646 0.29971498 0.82215844 0.52263086]
 [0.29971498 0.86416746 0.52263086 0.59655656]
 [0.86416746 0.04815052 0.59655656 0.31322287]
 [0.04815052 0.66181075 0.31322287 0.02934578]
 [0.05338908 0.3669698  0.86376853 0.99019456]
 [0.3669698  0.37207691 0.99019456 0.26679653]
 [0.37207691 0.82215844 0.26679653 0.97749604]
 [0.82215844 0.52263086 0.97749604 0.8854396 ]
 [0.52263086 0.59655656 0.8854396  0.91710142]
 [0.59655656 0.31322287 0.91710142 0.12305989]
 [0.31322287 0.02934578 0.12305989 0.24030879]
 [0.86376853 0.99019456 0.67939017 0.27623327]
 [0.99019456 0.26679653 0.27623327 0.88779469]
 [0.26679653 0.97749604 0.88779469 0.70786682]
 [0.97749604 0.8854396  0.70786682 0.40738597]
 [0.8854396  0.91710142 0.40738597 0.31880154]
 [0.91710142 0.12305989 0.31880154 0.62366982]
 [0.12305989 0.24030879 0.62366982 0.52629316]
 [0.67939017 